[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jvdzwaan/ocrpostcorrection/blob/main/colab/icdar-task1-hf-evaluation.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive 
drive.mount('/mntDrive')

Mounted at /mntDrive


In [ ]:
!git clone https://github.com/jvdzwaan/ocrpostcorrection.git

Cloning into 'ocrpostcorrection'...
remote: Enumerating objects: 480, done.
remote: Counting objects: 100% (480/480), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 480 (delta 271), reused 403 (delta 197), pack-reused 0
Receiving objects: 100% (480/480), 908.12 KiB | 8.18 MiB/s, done.
Resolving deltas: 100% (271/271), done.


In [ ]:
!pip install ./ocrpostcorrection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ocrpostcorrection
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 365 kB 13.9 MB/s 
     |████████████████████████████████| 305 kB 67.7 MB/s 
     |████████████████████████████████| 58 kB 4.1 MB/s 
     |████████████████████████████████| 4.9 MB 62.5 MB/s 
     |████████████████████████████████| 120 kB 72.5 MB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 115 kB 67.4 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
     |██████

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_from_disk

#icdar_dataset = load_from_disk('/mntDrive/MyDrive/icdar-seq_len-150')
#icdar_dataset = load_from_disk('../../data/ocrpostcorrection/icdar-seq_len-150')
icdar_dataset = load_from_disk('/Users/janneke/Google Drive/icdar-seq_len-150')

In [ ]:
for split in icdar_dataset.keys():
    icdar_dataset[split] = icdar_dataset[split].select(range(5))

In [ ]:
#model_dir = '/mntDrive/MyDrive/results-seq_len-150-0.3'
model_dir = '/Users/janneke/Google Drive/results-seq_len-150-0.3'
#model_dir = '/Users/janneke/models/results-seq_len-150-0.3'
model_name = 'bert-base-multilingual-cased'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from ocrpostcorrection.token_classification import tokenize_and_align_labels

tokenized_icdar = icdar_dataset.map(tokenize_and_align_labels(tokenizer), batched=True)

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=model_dir,          # output directory
    evaluation_strategy="epoch",
    num_train_epochs=3,
)

model = AutoModelForTokenClassification.from_pretrained(model_dir, num_labels=3)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_icdar['train'],         # training dataset
    eval_dataset=tokenized_icdar['val'],            # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
pred = trainer.predict(tokenized_icdar['test'])

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: key, start_token_id, tags, tokens, language, score. If key, start_token_id, tags, tokens, language, score are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7010
  Batch size = 8
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
pred.metrics

{'test_loss': 0.4178357720375061,
 'test_runtime': 108.8077,
 'test_samples_per_second': 64.426,
 'test_steps_per_second': 8.06}

In [ ]:
from pathlib import Path

#out_dir = Path('/mntDrive/MyDrive/results/icdar-seq_len-150')
out_dir = Path('/Users/janneke/Google Drive/results/icdar-seq_len-150')
out_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# Save predictions
import numpy as np

out_file = out_dir/'predictions'

np.save(out_file, pred.predictions)

# Can be run locally

In [ ]:
# Load predictions
import numpy as np

in_file = out_dir/'predictions.npy'

predictions = np.load(in_file)

In [ ]:
! cp -r /mntDrive/MyDrive/data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish .

In [ ]:
! ls

ICDAR2019_POCR_competition_evaluation_4M_without_Finnish  sample_data
ocrpostcorrection


In [ ]:
from ocrpostcorrection.icdar_data import generate_data

#in_dir = Path('ICDAR2019_POCR_competition_evaluation_4M_without_Finnish')
in_dir = Path('../../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish')
data_test, X_test = generate_data(in_dir)

5it [00:00,  5.12it/s]2022-09-18 12:13:35.457 | WARNING  | ocrpostcorrection.icdar_data:process_text:185 - UnicodeEncodeError for text ../../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/BG/BG1/47.txt; setting score to 1
11it [00:08,  1.22it/s]


In [ ]:
from ocrpostcorrection.utils import predictions2icdar_output, predictions_to_labels

output = predictions2icdar_output(tokenized_icdar['test'],
                                  predictions_to_labels(predictions),
                                  tokenizer,
                                  data_test)

2022-09-18 15:13:39.329 | WARNING  | ocrpostcorrection.utils:labels2label_str:78 - Missing predictions for token 140 in "NL/NL1/15.txt"
2022-09-18 15:13:39.329 | WARNING  | ocrpostcorrection.utils:labels2label_str:78 - Missing predictions for token 141 in "NL/NL1/15.txt"
2022-09-18 15:13:39.330 | WARNING  | ocrpostcorrection.utils:labels2label_str:78 - Missing predictions for token 142 in "NL/NL1/15.txt"
2022-09-18 15:13:39.330 | WARNING  | ocrpostcorrection.utils:labels2label_str:78 - Missing predictions for token 143 in "NL/NL1/15.txt"
2022-09-18 15:13:39.331 | WARNING  | ocrpostcorrection.utils:labels2label_str:78 - Missing predictions for token 144 in "NL/NL1/15.txt"
2022-09-18 15:13:39.331 | WARNING  | ocrpostcorrection.utils:labels2label_str:78 - Missing predictions for token 145 in "NL/NL1/15.txt"
2022-09-18 15:13:39.332 | WARNING  | ocrpostcorrection.utils:labels2label_str:78 - Missing predictions for token 146 in "NL/NL1/15.txt"
2022-09-18 15:13:39.332 | WARNING  | ocrpostcorr

In [ ]:
import json

out_file = out_dir/'results_task1_unfiltered_test_data.json'

with open(out_file, 'w') as f:
    json.dump(output, f)

In [ ]:
csv_file = out_file.with_suffix('.csv')

In [ ]:
from ocrpostcorrection.utils import runEvaluation

runEvaluation(in_dir, out_file, csv_file)

File	NbTokens	NbErroneousTokens	NbSymbolsConsidered	T1_Precision	T1_Recall	T1_Fmesure	T2_AvgLVDistOriginal	T2_AvgLVDistCorrected
SL/SL1/29.txt	2	2	532	1.00	0.79	0.88	0.99	0.99
SL/SL1/15.txt	83	28	405	0.72	0.84	0.78	0.15	0.15
SL/SL1/14.txt	115	50	626	0.94	0.82	0.88	0.15	0.15
SL/SL1/28.txt	273	21	1270	1.00	0.77	0.87	0.03	0.03
SL/SL1/16.txt	1004	182	4839	0.86	0.56	0.68	0.08	0.08
SL/SL1/17.txt	1009	333	4930	0.81	0.73	0.77	0.19	0.19
SL/SL1/13.txt	139	49	715	0.64	0.88	0.74	0.09	0.09
SL/SL1/12.txt	333	30	1491	0.97	0.69	0.81	0.03	0.03
SL/SL1/10.txt	314	36	1279	0.66	0.58	0.62	0.05	0.05
SL/SL1/38.txt	405	33	1995	0.68	0.59	0.63	0.03	0.03
SL/SL1/39.txt	143	73	696	0.72	0.81	0.76	0.15	0.15
SL/SL1/11.txt	238	19	1127	0.67	0.55	0.60	0.03	0.03
SL/SL1/9.txt	214	100	1048	0.82	0.76	0.79	0.17	0.17
SL/SL1/8.txt	1140	99	5728	0.69	0.51	0.59	0.04	0.04
SL/SL1/5.txt	263	15	1041	0.93	0.68	0.79	0.02	0.02
SL/SL1/43.txt	172	56	884	0.63	0.91	0.75	0.09	0.09
SL/SL1/42.txt	70	31	300	0.73	0.94	0.82	0.19	0.19
SL/SL1/4.txt	

In [ ]:
from ocrpostcorrection.utils import aggregate_results

results = aggregate_results(csv_file)
print(results.to_markdown())

| language   |   T1_Precision |   T1_Recall |   T1_Fmesure |
|:-----------|---------------:|------------:|-------------:|
| BG         |       0.85     |    0.693673 |     0.744286 |
| CZ         |       0.808043 |    0.623261 |     0.685652 |
| DE         |       0.971874 |    0.954152 |     0.962806 |
| EN         |       0.823333 |    0.618125 |     0.668333 |
| ES         |       0.8722   |    0.52     |     0.6254   |
| FI         |       0.896625 |    0.785625 |     0.833375 |
| FR         |       0.797703 |    0.571588 |     0.651368 |
| NL         |       0.875102 |    0.634286 |     0.690204 |
| PL         |       0.8872   |    0.7466   |     0.8026   |
| SL         |       0.806667 |    0.653333 |     0.692917 |


In [ ]:
print(results[['T1_Fmesure']].round(2).transpose().to_markdown())

|            |   BG |   CZ |   DE |   EN |   ES |   FI |   FR |   NL |   PL |   SL |
|:-----------|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|-----:|
| T1_Fmesure | 0.74 | 0.69 | 0.96 | 0.67 | 0.63 | 0.83 | 0.65 | 0.69 |  0.8 | 0.69 |
